# Lesson 1 - Semantic Search

### Import the Needed Packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

from DLAIUtils import Utils
import DLAIUtils

import os
import time
import torch

In [3]:
from pinecone import Pinecone, ServerlessSpec

In [4]:
from tqdm.auto import tqdm

In [5]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

### Load the Dataset

In [6]:
dataset = load_dataset('quora', split='train[240000:290000]')

Found cached dataset quora (/home/mark/.cache/huggingface/datasets/quora/default/0.0.0/36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04)


In [7]:
dataset[:5]

{'questions': [{'id': [207550, 351729],
   'text': ['What is the truth of life?', "What's the evil truth of life?"]},
  {'id': [33183, 351730],
   'text': ['Which is the best smartphone under 20K in India?',
    'Which is the best smartphone with in 20k in India?']},
  {'id': [351731, 351732],
   'text': ['Steps taken by Canadian government to improve literacy rate?',
    'Can I send homemade herbal hair oil from India to US via postal or private courier services?']},
  {'id': [37799, 94186],
   'text': ['What is a good way to lose 30 pounds in 2 months?',
    'What can I do to lose 30 pounds in 2 months?']},
  {'id': [351733, 351734],
   'text': ['Which of the following most accurately describes the translation of the graph y = (x+3)^2 -2 to the graph of y = (x -2)^2 +2?',
    'How do you graph x + 2y = -2?']}],
 'is_duplicate': [False, True, False, True, False]}

In [8]:
# Extract questions and drop ids
questions = []
for record in dataset['questions']:
    questions.extend(record['text'])
# Remove duplicate questions
question = list(set(questions))
print('\n'.join(questions[:10]))
print('-' * 50)
print(f'Number of questions: {len(questions)}')
print(f'Number of unique questions: {len(question)}')

What is the truth of life?
What's the evil truth of life?
Which is the best smartphone under 20K in India?
Which is the best smartphone with in 20k in India?
Steps taken by Canadian government to improve literacy rate?
Can I send homemade herbal hair oil from India to US via postal or private courier services?
What is a good way to lose 30 pounds in 2 months?
What can I do to lose 30 pounds in 2 months?
Which of the following most accurately describes the translation of the graph y = (x+3)^2 -2 to the graph of y = (x -2)^2 +2?
How do you graph x + 2y = -2?
--------------------------------------------------
Number of questions: 100000
Number of unique questions: 88919


### Check cuda and Setup the model

**Note**: "Checking cuda" refers to checking if you have access to GPUs (faster compute). In this course, we are using CPUs. So, you might notice some code cells taking a little longer to run.

We are using *all-MiniLM-L6-v2* sentence-transformers model that maps sentences to a 384 dimensional dense vector space.

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print('Sorry no cuda.')
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

Sorry no cuda.


In [10]:
query = 'which city is the most populated in the world?'
xq = model.encode(query)
xq.shape

(384,)

### Setup Pinecone

In [16]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
pinecone = Pinecone(api_key=PINECONE_API_KEY)
print(pinecone.list_indexes())

{'indexes': [{'dimension': 384,
              'host': 'openai-labs-384-djnczx7.svc.apw5-4e34-81fa.pinecone.io',
              'metric': 'cosine',
              'name': 'openai-labs-384',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 8,
              'host': 'quickstart-djnczx7.svc.apw5-4e34-81fa.pinecone.io',
              'metric': 'cosine',
              'name': 'quickstart',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 512,
              'host': 'openai-labs-512-djnczx7.svc.apw5-4e34-81fa.pinecone.io',
              'metric': 'dotproduct',
              'name': 'openai-labs-512',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 153

In [12]:
INDEX_NAME="openai-labs-384"

if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
    pinecone.delete_index(INDEX_NAME)
print(INDEX_NAME)
pinecone.create_index(name=INDEX_NAME, 
    dimension=model.get_sentence_embedding_dimension(), 
    metric='cosine',
    spec=ServerlessSpec(cloud='aws', region='us-west-2'))

index = pinecone.Index(INDEX_NAME)
print(index)

openai-labs-384


### Create Embeddings and Upsert to Pinecone

In [13]:
batch_size=200
vector_limit=10000

questions = question[:vector_limit]

import json

range(0, len(questions), batch_size)

i = 0
i_end = 200
ids = [str(x) for x in range(i, i_end)]
print(ids)
metadatas = [{'text': text} for text in questions[i:i_end]]
print(metadatas)
xc = model.encode(questions[i:i_end])
print(xc)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '15

In [14]:
batch_size=200
vector_limit=10000

questions = question[:vector_limit]

import json

for i in tqdm(range(0, len(questions), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(questions))
    # create IDs batch
    ids = [str(x) for x in range(i, i_end)]
    # create metadata batch
    metadatas = [{'text': text} for text in questions[i:i_end]]
    # create embeddings
    xc = model.encode(questions[i:i_end])
    # create records list for upsert
    records = zip(ids, xc, metadatas)
    # upsert to Pinecone
    index.upsert(vectors=records)

  0%|          | 0/50 [00:00<?, ?it/s]

In [15]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 9800}},
 'total_vector_count': 9800}

### Run Your Query

In [17]:
# small helper function so we can repeat queries later
def run_query(query):
  embedding = model.encode(query).tolist()
  results = index.query(top_k=10, vector=embedding, include_metadata=True, include_values=False)
  for result in results['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

In [18]:
run_query('which city has the highest population in the world?')

0.61: What is the most modern city in China?
0.58: What is the largest race of people on Earth?
0.58: What is the biggest country?
0.56: What are the largest cities in Spain?
0.54: What are the 20 most richest countries in the world?
0.53: Which is the biggest arena in the world?
0.53: Which is the top worst country in the world?
0.52: Which are the worst cities of India?
0.51: What are the largest cities in Spain and what are they most known for?
0.51: What do you think is the greatest country in the world?


In [19]:
query = 'how do i make chocolate cake?'
run_query(query)

0.81: How can I make a delicious cake?
0.6: How do I bake a cake without an oven?
0.57: How do you make candles?
0.56: How do you bake cakes in a convection oven?
0.55: How do I make whipped cream?
0.51: How do we bake cake in microwave oven? What should be the perfect temperature?
0.5: How do I make a red fondant?
0.49: Are You Looking For Tasty Chocolates in Bangalore?
0.49: How can I make a banana pudding without bananas?
0.47: What is the recipe for a spinach artichoke dip from the Cheesecake Factory?
